In [ ]:
import asf_search as asf
import os
import zipfile
from pyroSAR import identify
from spatialist import Raster
import numpy as np
import matplotlib.pyplot as plt

results = asf.search(
    platform=[asf.PLATFORM.SENTINEL1],
    processingLevel=[asf.PRODUCT_TYPE.GRD_HD],
    start="2022-01-01",
    end="2022-01-02",
    intersectsWith="POLYGON((-120 35, -120 36, -119 36, -119 35, -120 35))"
)

print("Found scenes:", len(results))
results[0].download(path="./data")


["'type': 'REVERSE': 'report': Reversed polygon winding order"]


Found scenes: 3


/opt/miniconda3/envs/photo/lib/python3.10/site-packages/asf_search/download/download.py:66: UserWarning: File already exists, skipping download: ./data/S1A_IW_GRDH_1SDV_20220101T135219_20220101T135244_041268_04E7A6_6B2B.zip
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')


In [ ]:
data_dir = "./data"
output_dir = "./eda_results"
os.makedirs(output_dir, exist_ok=True)

# loop over all downloaded zip files
for f in os.listdir(data_dir):
    if f.endswith(".zip"):
        path = os.path.join(data_dir, f)

        # identify metadata
        meta = identify(path)
        print(f"Scene: {f}")
        print(meta)

        # read VV band
        vv_files = meta.findfiles('^measurement.*vv.*tiff$')
        if not vv_files:
            continue
        vv_path = vv_files[0]

        with Raster(vv_path) as ras:
            arr = ras.array()
            arr = arr.astype(np.float32)

            # statistics
            print("Shape:", arr.shape)
            print("Min:", np.min(arr), "Max:", np.max(arr))
            print("Mean:", np.mean(arr), "Std:", np.std(arr))

            # save histogram
            plt.figure()
            plt.hist(arr.flatten(), bins=200, range=(np.min(arr), np.max(arr)))
            plt.title(f"Histogram VV - {f}")
            plt.xlabel("Backscatter Amplitude")
            plt.ylabel("Frequency")
            plt.savefig(os.path.join(output_dir, f"{f}_histogram.png"))
            plt.close()

            # save quick image preview
            plt.figure(figsize=(10,10))
            plt.imshow(arr, cmap='gray')
            plt.title(f"VV Preview - {f}")
            plt.colorbar(label="Amplitude")
            plt.savefig(os.path.join(output_dir, f"{f}_preview.png"))
            plt.close()


Scene: S1A_IW_GRDH_1SDV_20220101T135219_20220101T135244_041268_04E7A6_6B2B.zip
pyroSAR ID object of type SAFE
acquisition_mode: IW
coordinates: [(-116.354172, 34.18055), (-119.091644, 34.583302), (-118.768211, 36.083237), (-115.979294, 35.681915)]
cycleNumber: 250
frameNumber: 321446
lines: 16716
orbit: D
orbitNumber_abs: 41268
orbitNumber_rel: 71
polarizations: ['VV', 'VH']
product: GRD
projection: +proj=longlat +datum=WGS84 +no_defs
samples: 25576
sensor: S1A
spacing: (10.0, 10.0)
start: 20220101T135219
stop: 20220101T135244


In [ ]:
data_dir = "./data"
output_dir = "./eda_results"
os.makedirs(output_dir, exist_ok=True)

for fname in os.listdir(data_dir):
    if not fname.endswith(".zip"):
        continue

    zip_path = os.path.join(data_dir, fname)
    meta = identify(zip_path)
    print(f"\nScene: {fname}\n{meta}")

    extract_dir = os.path.join(data_dir, fname + "_extracted")
    if not os.path.exists(extract_dir):
        with zipfile.ZipFile(zip_path) as zf:
            zf.extractall(extract_dir)

    vv_files = [
        os.path.join(root, file)
        for root, _, files in os.walk(extract_dir)
        for file in files
        if file.lower().endswith(".tiff") and "vv" in file.lower()
    ]
    if not vv_files:
        print("No VV band found, skipping.")
        continue

    vv_path = os.path.abspath(vv_files[0])
    with Raster(vv_path, list_separate=False) as ras:
        vv_arr = ras.array().astype(np.float32)

    stats = {
        "shape": vv_arr.shape,
        "min": float(np.min(vv_arr)),
        "max": float(np.max(vv_arr)),
        "mean": float(np.mean(vv_arr)),
        "std": float(np.std(vv_arr)),
    }
    print(stats)

    plt.figure()
    plt.hist(vv_arr.ravel(), bins=200)
    plt.title(f"Histogram VV - {fname}")
    plt.xlabel("Backscatter Amplitude")
    plt.ylabel("Frequency")
    plt.savefig(os.path.join(output_dir, f"{fname}_VV_histogram.png"))
    plt.close()

    plt.figure(figsize=(10, 10))
    plt.imshow(vv_arr, cmap="gray")
    plt.title(f"VV Preview - {fname}")
    plt.colorbar(label="Amplitude")
    plt.savefig(os.path.join(output_dir, f"{fname}_VV_preview.png"))
    plt.close()
